In [1]:
!pip install rank_bm25

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!tar -xzf /content/drive/MyDrive/Isabelle2024_linux.tar.gz -C /content/
print('Extraction finished.')

Extraction finished.


In [4]:
import os
os.environ['ISABELLE_DIRPATH'] = os.path.abspath('/content/Isabelle2024')

import nest_asyncio
nest_asyncio.apply()

In [5]:
!wget https://github.com/leanprover/elan/releases/latest/download/elan-x86_64-unknown-linux-gnu.tar.gz
!tar -xzf elan-x86_64-unknown-linux-gnu.tar.gz
!./elan-init -y --no-modify-path --default-toolchain leanprover/lean4:stable
import os
os.environ["PATH"] += ":/root/.elan/bin"
os.environ["ELAN_HOME"] = "/root/.elan/"

--2025-07-03 20:50:37--  https://github.com/leanprover/elan/releases/latest/download/elan-x86_64-unknown-linux-gnu.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/leanprover/elan/releases/download/v4.1.2/elan-x86_64-unknown-linux-gnu.tar.gz [following]
--2025-07-03 20:50:37--  https://github.com/leanprover/elan/releases/download/v4.1.2/elan-x86_64-unknown-linux-gnu.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/128052474/1ed74d07-be3b-45de-8509-ada4de88f33e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250703%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250703T205037Z&X-Amz-Expires=1800&X-Amz-Signature=ca9d524d8214f68c62b32499dd3a3f5b122eab5b01237a4c89375c075

In [6]:
!git clone https://github.com/leanprover-community/repl.git
%cd repl
!echo '' >> lakefile.toml
!echo '[[require]]' >> lakefile.toml
!echo 'name = "mathlib"' >> lakefile.toml
!echo 'scope = "leanprover-community"' >> lakefile.toml
!lake build repl
!lake update
!echo '{"path": "test/file.lean"}' | lake exe repl
%cd ..
import os
os.environ['REPL_PATH'] = os.path.abspath('/content/repl')

fatal: destination path 'repl' already exists and is not an empty directory.
/content/repl
Build completed successfully.
info: toolchain not updated; already up-to-date
info: mathlib: running post-update hooks
Current branch: HEAD
`git for-each-ref --contains 2731c229b7f2149fce47a8714cbb04190912d8b9 refs/remotes/origin/pr/* --format=%(refname)` returned:
         with exit code 0 and stderr: .
`git for-each-ref --contains 2731c229b7f2149fce47a8714cbb04190912d8b9 refs/remotes/origin/pr/* --format="%(refname)"` returned:
         with exit code 0 and stderr: .
Using cache from origin: leanprover-community/mathlib4
No files to download
Decompressing 6909 file(s)
Unpacked in 159 ms
Completed successfully!
info: mathlib: running post-update hooks
Current branch: HEAD
`git for-each-ref --contains 2731c229b7f2149fce47a8714cbb04190912d8b9 refs/remotes/origin/pr/* --format=%(refname)` returned:
         with exit code 0 and stderr: .
`git for-each-ref --contains 2731c229b7f2149fce47a8714cbb0419

In [7]:
!git clone https://github.com/lanzhang128/multi_agent_autoformalization.git
%cd multi_agent_autoformalization
!mkdir test_results
!chmod +x theorem_prover/repl.sh

fatal: destination path 'multi_agent_autoformalization' already exists and is not an empty directory.
/content/multi_agent_autoformalization
mkdir: cannot create directory ‘test_results’: File exists


#Settings

###Prepare test example and few-shot examplars

This is the test natural language mathematical statement from miniF2F.

In [8]:
informal_statement = "Given a [[nonnegative]] real number $x$, let $\\langle x\\rangle$ denote the fractional part of $x$; that is, $\\langle x\\rangle=x-\\lfloor x\\rfloor$, where $\\lfloor x\\rfloor$ denotes the [[greatest integer]] less than or equal to $x$. Suppose that $a$ is positive, $\\langle a^{-1}\\rangle=\\langle a^2\\rangle$, and $2<a^2<3$. Find the value of $a^{12}-144a^{-1}$. Show that it is 233."

The autoformalization agent supports the generation of formalization code via an LLM, given a natural language mathematical statement and optional informal-formal statement pairs as exemplars. Here we prepare three examplars for both Isabelle/HOL and Lean4.

In [9]:
isabelle_pairs = [
    [
        "Given that\n\n$$\n\\begin{align*}x_{1}&=211,\\\\\nx_{2}&=375,\\\\\nx_{3}&=420,\\\\\nx_{4}&=523,"
        "\\ \\text{and}\\\\\nx_{n}&=x_{n-1}-x_{n-2}+x_{n-3}-x_{n-4}\\ \\text{when}\\ n\\geq5, "
        "\\end{align*}\n$$\n\nfind the value of $x_{531}+x_{753}+x_{975}$. Show that it is 898.",
        "theory aimeII_2001_p3 imports\n  Complex_Main\nbegin\n\ntheorem aimeII_2001_p3:\n  fixes x :: \"nat "
        "\\<Rightarrow> int\"\n  assumes h0 : \"x 1 = 211\"\n    and h1 : \"x 2 = 375\"\n    and h2 : \"x 3 = "
        "420\"\n    and h3 : \"x 4 = 523\"\n    and h4 : \"\\<And>(n::nat). ((n\\<ge>5) \\<Longrightarrow> (x "
        "n = x (n-1) - x (n-2) + x (n-3) - x (n-4)))\"\n  shows \"x 531 + x 753 + x 975 = 898\"\n  sorry\n\nend"
    ],
    [
        "Show that for any complex number $x$, $x^2 + 49 = (x + 7i)(x - 7i)$.",
        "theory algebra_2complexrootspoly_xsqp49eqxp7itxpn7i imports\n  Complex_Main\nbegin\n\ntheorem "
        "algebra_2complexrootspoly_xsqp49eqxp7itxpn7i:\n  fixes x :: complex\n  shows \"x^2 + 49 = (x + 7 * "
        "\\<i>) * (x - 7 * \\<i>)\"\n  sorry\n\nend"
    ],
    [
        "Compute the sum of all the roots of\n$(2x+3)(x-4)+(2x+3)(x-6)=0 $\n\n$ \\textbf{(A) } \\frac{7}{"
        "2}\\qquad \\textbf{(B) } 4\\qquad \\textbf{(C) } 5\\qquad \\textbf{(D) } 7\\qquad \\textbf{(E) } 13 "
        "$ Show that it is \\textbf{(A) }7/2.",
        "theory amc12a_2002_p1 imports\n  Complex_Main\n  "
        "\"HOL-Computational_Algebra.Computational_Algebra\"\n  \nbegin\n\ntheorem amc12a_2002_p1:\n  fixes "
        "f::\"complex \\<Rightarrow> complex\"\n  assumes \"\\<forall> x. f x = (2 * x + 3) * (x - 4) + (2 * "
        "x + 3) * (x - 6)\"\n  shows \"(\\<Sum> y \\<in> f -` {0}. y) = 7/2\"\n  sorry\n\nend"
    ]
]
lean4_pairs = [
    [
        "Given that\n\n$$\n\\begin{align*}x_{1}&=211,\\\\\nx_{2}&=375,\\\\\nx_{3}&=420,\\\\\nx_{4}&=523,"
        "\\ \\text{and}\\\\\nx_{n}&=x_{n-1}-x_{n-2}+x_{n-3}-x_{n-4}\\ \\text{when}\\ n\\geq5, "
        "\\end{align*}\n$$\n\nfind the value of $x_{531}+x_{753}+x_{975}$. Show that it is 898.",
        "import Mathlib\n\nopen BigOperators Real Nat Topology\n\ntheorem aimeII_2001_p3\n  (x : ℕ → ℤ)\n  (h₀ : "
        "x 1 = 211)\n  (h₂ : x 2 = 375)\n  (h₃ : x 3 = 420)\n  (h₄ : x 4 = 523)\n  (h₆ : ∀ n ≥ 5, x n = x (n - 1) "
        "- x (n - 2) + x (n - 3) - x (n - 4)) :\n  x 531 + x 753 + x 975 = 898 := sorry"
    ],
    [
        "Show that for any complex number $x$, $x^2 + 49 = (x + 7i)(x - 7i)$.",
        "import Mathlib\n\nopen BigOperators Real Nat Topology\n\ntheorem "
        "algebra_2complexrootspoly_xsqp49eqxp7itxpn7i\n  (x : ℂ) :\n  x^2 + 49 = (x + (7 * Complex.I)) * (x + (-7 "
        "* Complex.I)) := sorry"
    ],
    [
        "Compute the sum of all the roots of\n$(2x+3)(x-4)+(2x+3)(x-6)=0 $\n\n$ \\textbf{(A) } \\frac{7}{"
        "2}\\qquad \\textbf{(B) } 4\\qquad \\textbf{(C) } 5\\qquad \\textbf{(D) } 7\\qquad \\textbf{(E) } 13 $ "
        "Show that it is \\textbf{(A) }7/2.",
        "import Mathlib\n\nopen BigOperators Real Nat Topology\n\ntheorem amc12a_2002_p1\n  (f : ℂ → ℂ)\n  (h₀ : "
        "∀ x, f x = (2 * x + 3) * (x - 4) + (2 * x + 3) * (x - 6))\n  (h₁ : Fintype (f ⁻¹' {0})) :\n  ∑ y in ("
        "f⁻¹' {0}).toFinset, y = 7 / 2 := sorry"
    ]
]

###Instantiate agents

We present the iterative self-refinement multi-agent system that leverages an autoformalization agent, a hard-critique agent, a soft-critique agent, a formal refinement agent, and an informal refinement agent to iteratively improve autoformalization performance using a same backend LLM. We first instantiate an LLM.

In [10]:
from llm import OpenAILLM
gpt_mini = OpenAILLM(
    api_key='api-key',
    model='gpt-4.1-mini')

We then instantiate autoformalization agents.

In [11]:
from agent import AutoformalizationAgent
afa_isabelle = AutoformalizationAgent(llm=gpt_mini, formal_language='Isabelle/HOL')
afa_lean = AutoformalizationAgent(llm=gpt_mini, formal_language='Lean4')

We then instantiate critique agents. The critique agents provide critiques of formalized codes from specific aspects. They are divided into two categories: (1) **Hard**: critiques are produced by the relevant theorem prover to assess the syntactic aspects of the formalization; (2) **Soft**: soft-critique agents follow the concept of LLM-as-a-Judge. For soft-critique agent, we focus on whether the formalized code involves all mathematical concepts in the natural language statement.

In [12]:
from agent import HardCritiqueAgent, SoftCritiqueAgent

hca_isabelle = HardCritiqueAgent(formal_language='Isabelle/HOL', file_dir='./test_results')
hca_lean = HardCritiqueAgent(formal_language='Lean4', file_dir='./test_results')

aspect_description = 'whether the formalized code involves all mathematical concepts in the natural language statement.'
sca_isabelle = SoftCritiqueAgent(llm=gpt_mini, formal_language='Isabelle/HOL', aspect_description=aspect_description)
sca_lean = SoftCritiqueAgent(llm=gpt_mini, formal_language='Lean4', aspect_description=aspect_description)

logging file is at: ./test_results/isabelle.log
Isabelle server started with info: server "test" = 127.0.0.1:40771 (password "7e18da90-e1d5-4313-b437-976fe11cf797")
 in 2.01s.
Isabelle server session 343a2893-34d0-4ca0-80cb-88eea3d03db6 started in 12.36s.
logging file is at: ./test_results/lean.log


We then instantiate refinement agents. Refinement agents refine the formalization based on given feedback. Depending on the source of the feedback, they are divided into two categories: (1) **Formal**: the agent refines the formal codes from a syntactic perspective, given the hard critiques from the relevant theorem prover; (2) **Informal**: the agent refines the formal codes based on the soft critiques from an LLM.

In [13]:
from agent import FormalRefinementAgent, InformalRefinementAgent

fra_isabelle = FormalRefinementAgent(llm=gpt_mini, formal_language='Isabelle/HOL')
fra_lean = FormalRefinementAgent(llm=gpt_mini, formal_language='Lean4')

ira_isabelle = InformalRefinementAgent(llm=gpt_mini, formal_language='Isabelle/HOL')
ira_lean = InformalRefinementAgent(llm=gpt_mini, formal_language='Lean4')

#Iterative Self-refinement Multi-agent System

We use established agents for iterative improving the autoformalization process.

### Isabelle/HOL

We first perform zero-shot autoformalization.

In [14]:
formal, _ = afa_isabelle(informal_statement=informal_statement)
print(formal)

theory Fractional_Part_Problem
  imports Complex_Main
begin

definition frac :: "real ⇒ real" where
  "frac x = x - floor x"

lemma problem:
  fixes a :: real
  assumes "a > 0"
    and "frac (1 / a) = frac (a^2)"
    and "2 < a^2" "a^2 < 3"
  shows "a^12 - 144 * a^(-1) = 233"
  sorry

end



Then we perform few-shot autoformalization.

In [15]:
formal, _ = afa_isabelle(informal_statement=informal_statement, informal_formal_pairs=isabelle_pairs)
print(formal)

theory fractional_part_problem imports
  Complex_Main
begin

definition frac :: "real \<Rightarrow> real" where
  "frac x = x - floor x"

theorem fractional_part_problem:
  fixes a :: real
  assumes "a > 0"
    and "frac (1 / a) = frac (a^2)"
    and "2 < a^2" "a^2 < 3"
  shows "a^12 - 144 * a^(-1) = 233"
  sorry

end



We use the hard-critique agent to check whether the formalization is syntactically correct.

In [16]:
correctness, error_details = hca_isabelle(formalization=formal, file_prefix='isabelle_gpt_0')
print(correctness)
print(error_details)

Isabelle server use_theory ended in 2.41s.
False
Identified error on line: 9
Error message: Type unification failed: No type arity nat :: uminus

Type error in application: incompatible operand type

Operator:  (^) a :: nat \<Rightarrow> real
Operand:   - 1 :: ??'a

Coercion Inference:

Local coercion insertion on the operand failed:
No type arity nat :: uminus

Now trying to infer coercions globally.

Coercion inference failed:
no infimum

Cannot fulfil subtype constraints:
  nat  <:  nat   from function application  a\<^sup>2
  nat  <:  nat   from function application  a\<^sup>2
  nat  <:  nat   from function application  a\<^sup>2
  nat  <:  nat   from function application  a ^ 12
  ??'a  <:  nat   from function application  a ^ - 1
  ??'a  <:  ??'b   from function application  - 1

Identified error on line: 15
Error message: Bad context for command "sorry"\<^here> -- using reset state




We use the soft-critique agent to check whether the formalization is semantically correct.

In [17]:
judgement, _= sca_isabelle(informal_statement=informal_statement, formalization=formal)
print(judgement)

Explanation: The formalization includes all the mathematical concepts mentioned in the natural language statement. It defines the fractional part function frac x = x - floor x, explicitly capturing the fractional part notation ⟨x⟩. It assumes a is positive, includes the condition ⟨a^{-1}⟩ = ⟨a^2⟩, and the inequalities 2 < a^2 < 3. Finally, it states the goal to prove a^{12} - 144 a^{-1} = 233. Thus, all key concepts—nonnegative real number (a > 0), fractional part definition, greatest integer function (floor), the equality of fractional parts, the inequality bounds on a^2, and the expression to be evaluated—are present in the formalization.

Judgement: True



We define a function to extract the judgement value.

In [18]:
import re
def judge_value(text):
  judgement = re.findall('Judgement: (True|False)', text, flags=re.DOTALL)
  if judgement:
    correctness = judgement[-1].split()[-1]
    if correctness == 'True':
      return True
    else:
      return False
  else:
    return None

We iteratively refine the formalization. For each iteration, we first check whether it is syntactically correct. If it is incorrect, then we use formal refinement agent to refine. If it is correct, then we check whether it is semantically correct. If it is incorrect, then we use informal refinement agent to refine.

In [19]:
for j in range(4):
  if correctness != 'True':
    with open(f'./test_results/isabelle_gpt_{j}.thy', 'r', encoding='utf-8') as f:
      formalization = f.read()
    formal, _ = fra_isabelle(informal_statement=informal_statement,
                  refinement_mode='detailed',
                  formalization=formalization,
                  correctness=correctness,
                  error_details=error_details)
    correctness, error_details = hca_isabelle(formalization=formal, file_prefix=f'isabelle_gpt_{j + 1}')
    judgement, _ = sca_isabelle(informal_statement=informal_statement, formalization=formal)
  else:
    if not judge_value(judgement):
      formal, _ = ira_isabelle(informal_statement=informal_statement,
                    formalization=formal,
                    aspect_description=aspect_description,
                    aspect_evaluation=judgement)
      correctness, error_details = hca_isabelle(formalization=formal, file_prefix=f'isabelle_gpt_{j + 1}')
      judgement, _ = sca_isabelle(informal_statement=informal_statement, formalization=formal)
print('Formalization:\n', formal)
print('Correctness:\n', correctness)
print('Error details:\n', error_details)
print('Judgement:\n', judgement)

Isabelle server use_theory ended in 2.26s.
Formalization:
 theory isabelle_gpt_0
imports
  Complex_Main
begin

definition frac :: "real \<Rightarrow> real" where
  "frac x = x - floor x"

theorem fractional_part_problem:
  fixes a :: real
  assumes "a > 0"
    and "frac (1 / a) = frac (a^2)"
    and "2 < a^2" "a^2 < 3"
  shows "a^12 - 144 * (1 / a) = 233"
  sorry

end

Correctness:
 True
Error details:
 
Judgement:
 Explanation: The formalization includes all the mathematical concepts mentioned in the natural language statement. It defines the fractional part function `frac` as \( \langle x \rangle = x - \lfloor x \rfloor \), where `floor` is the greatest integer less than or equal to \( x \). The assumptions in the theorem correspond exactly to the conditions given: \( a > 0 \), \( \langle a^{-1} \rangle = \langle a^2 \rangle \), and \( 2 < a^2 < 3 \). The goal statement matches the expression to be proved, \( a^{12} - 144 a^{-1} = 233 \). Therefore, all mathematical concepts from the

### Lean4

We first perform zero-shot autoformalization.

In [20]:
formal, _ = afa_lean(informal_statement=informal_statement)
print(formal)

import Mathlib.Data.Real.Basic
import Mathlib.Data.Int.Basic
import Mathlib.Tactic.Basic

open Real Int

def fractional_part (x : ℝ) : ℝ := x - Int.floor x

theorem problem_statement (a : ℝ) (ha_pos : 0 < a) (h_frac_eq : fractional_part (a⁻¹) = fractional_part (a^2))
  (h_bound : 2 < a^2 ∧ a^2 < 3) : a^12 - 144 * a⁻¹ = 233 := by
  sorry



Then we perform few-shot autoformalization.

In [21]:
formal, _ = afa_lean(informal_statement=informal_statement, informal_formal_pairs=lean4_pairs)
print(formal)

import Mathlib.Analysis.SpecialFunctions.Floor

open Real

theorem fractional_part_problem
  (a : ℝ)
  (ha_pos : 0 < a)
  (ha2_range : 2 < a^2 ∧ a^2 < 3)
  (h_frac_eq : (a⁻¹ - Real.floor (a⁻¹)) = (a^2 - Real.floor (a^2))) :
  a^12 - 144 * a⁻¹ = 233 := sorry



We use the hard-critique agent to check whether the formalization is syntactically correct.

In [22]:
correctness, error_details = hca_lean(formalization=formal, file_prefix='lean_gpt_0')
print(correctness)
print(error_details)

False
Identified error on line: 3
Error message: unknown namespace 'Real'

Identified error on line: 7
Error message: expected token




We use the soft-critique agent to check whether the formalization is semantically correct.

In [23]:
judgement, _= sca_lean(informal_statement=informal_statement, formalization=formal)
print(judgement)

Explanation: The natural language statement involves several mathematical concepts: a nonnegative real number \( x \) and its fractional part \(\langle x \rangle = x - \lfloor x \rfloor\), where \(\lfloor x \rfloor\) is the greatest integer less than or equal to \( x \). It then specifies conditions on \( a \): \( a \) is positive, the fractional parts of \( a^{-1} \) and \( a^2 \) are equal, and \( 2 < a^2 < 3 \). The goal is to find the value of \( a^{12} - 144 a^{-1} \) and show it equals 233.

The Lean4 formalization explicitly includes:
- \( a : \mathbb{R} \) with \( a > 0 \) (ha_pos),
- the range \( 2 < a^2 < 3 \) (ha2_range),
- the equality of fractional parts expressed as \( a^{-1} - \lfloor a^{-1} \rfloor = a^2 - \lfloor a^2 \rfloor \) (h_frac_eq),
- and the conclusion \( a^{12} - 144 a^{-1} = 233 \).

The formalization uses the floor function from mathlib and directly encodes the fractional part as \( x - \lfloor x \rfloor \), matching the natural language definition. The con

We iteratively refine the formalization. For each iteration, we first check whether it is syntactically correct. If it is incorrect, then we use formal refinement agent to refine. If it is correct, then we check whether it is semantically correct. If it is incorrect, then we use informal refinement agent to refine.

In [24]:
for j in range(4):
  if correctness != 'True':
    with open(f'./test_results/lean_gpt_{j}.lean', 'r', encoding='utf-8') as f:
      formalization = f.read()
    formal, _ = fra_lean(informal_statement=informal_statement,
                refinement_mode='detailed',
                formalization=formalization,
                correctness=correctness,
                error_details=error_details)
    correctness, error_details = hca_lean(formalization=formal, file_prefix=f'lean_gpt_{j + 1}')
    judgement, _ = sca_lean(informal_statement=informal_statement, formalization=formal)
  else:
    if not judge_value(judgement):
      formal, _ = ira_lean(informal_statement=informal_statement,
                  formalization=formal,
                  aspect_description=aspect_description,
                  aspect_evaluation=judgement)
      correctness, error_details = hca_lean(formalization=formal, file_prefix=f'lean_gpt_{j + 1}')
      judgement, _ = sca_lean(informal_statement=informal_statement, formalization=formal)
print('Formalization:\n', formal)
print('Correctness:\n', correctness)
print('Error details:\n', error_details)
print('Judgement:\n', judgement)

Formalization:
 import Mathlib.Analysis.SpecialFunctions.Floor

open Real

theorem fractional_part_problem
  (a : ℝ)
  (ha_pos : 0 < a)
  (ha2_range : 2 < a^2 ∧ a^2 < 3)
  (h_frac_eq : (a⁻¹ - Real.floor (a⁻¹)) = (a^2 - Real.floor (a^2))) :
  a^12 - 144 * a⁻¹ = 233 := sorry

Correctness:
 False
Error details:
 Identified error on line: 3
Error message: unknown namespace 'Real'

Identified error on line: 7
Error message: expected token


Judgement:
 Explanation: The formalized code involves all the mathematical concepts present in the natural language statement. It includes the variable \(a\) as a real number, the positivity condition \(0 < a\), the range condition \(2 < a^2 < 3\), and the equality of fractional parts \(\langle a^{-1} \rangle = \langle a^2 \rangle\) expressed as \(a^{-1} - \lfloor a^{-1} \rfloor = a^2 - \lfloor a^2 \rfloor\). The code also states the conclusion to prove, \(a^{12} - 144 a^{-1} = 233\). The definition of fractional part and floor function is implicitly use

###Release resources

In [25]:
hca_isabelle.theorem_prover.terminate()

Isabelle server 127.0.0.1:40771 shutdown.
